# Смысл программы

Визуализация генеалогического дерева: парсинг файла на языке Prolog

# Библиотеки

In [ ]:
!pip install graphviz

In [ ]:
import re
from collections import defaultdict
from graphviz import Digraph

# Основная часть

In [ ]:
filename = 'new_family.pl'  # Изначальный файл на языке Prolog
save_filename = 'family_tree'  # Сохраняемый файл с генеалогическим деревом
save_format = 'png'  # Формат сохраняеемого файла

## Парсинг файла на языке Prolog

In [ ]:
def read_parent_file(filename):
    family_tree = defaultdict(list)
    marriages = []
    genders = {}  # Словарь для хранения пола

    with open(filename, 'r') as file:
        for line in file:

            # Извлечение x и y из parent(x, y)
            parent_match = re.match(r'parent\((\w+),\s*(\w+)\)', line.strip())
            if parent_match:
                parent, child = parent_match.groups()
                family_tree[parent].append(child)

            # Извлечение x и y из marry(x, y)
            marry_match = re.match(r'marry\((\w+),\s*(\w+)\)', line.strip())
            if marry_match:
                spouse1, spouse2 = marry_match.groups()
                marriages.append((spouse1, spouse2))

            # Извлечение пола
            gender_match = re.match(r'man\((\w+)\)', line.strip())
            if gender_match:
                genders[gender_match.group(1)] = 'male'
            else:
                gender_match = re.match(r'woman\((\w+)\)', line.strip())
                if gender_match:
                    genders[gender_match.group(1)] = 'female'

    return family_tree, marriages, genders

In [ ]:
def build_family_tree(family_tree):
    all_people = set(family_tree.keys())
    all_children = {child for children in family_tree.values() for child in children}

    root_people = all_people - all_children

    return root_people

## Построение графа

In [ ]:
def draw_family_tree(family_tree, root_people, marriages, genders):
    dot = Digraph(comment='Family Tree')

    # Добавление супружеских пар в отдельную подгруппу
    for spouse1, spouse2 in marriages:
        with dot.subgraph() as s:
            s.attr(rank='same')  # Все узлы в этой подгруппе будут на одном уровне
            s.node(spouse1, shape='doublecircle' if genders.get(spouse1) == 'male' else 'circle')
            s.node(spouse2, shape='doublecircle' if genders.get(spouse2) == 'male' else 'circle')
            s.edge(spouse1, spouse2, dir='none')  # Ненаправленная связь

    for parent, children in family_tree.items():
        shape = 'doublecircle' if genders.get(parent) == 'male' else 'circle'  # Определяем форму узла родителя
        dot.node(parent, shape=shape)  # Узел-родитель

        for child in children:
            child_shape = 'doublecircle' if genders.get(child) == 'male' else 'circle'
            dot.node(child, shape=child_shape)  # Узел-ребенок
            dot.edge(parent, child)  # Связь между родителем и ребенком

    # Сохраняем и отображаем граф
    dot.render(save_filename, format=save_format, cleanup=True)

## Запуск

In [ ]:
def main():
    family_tree, marriages, genders = read_parent_file(filename)
    root_people = build_family_tree(family_tree)
    draw_family_tree(family_tree, root_people, marriages, genders)
    print(f"Генеалогическое дерево сохранено как {save_filename}.{save_format}")

Генеалогическое дерево сохранено как 'family_tree.png'.


In [ ]:
if __name__ == "__main__":
    main()